In [110]:
import random
import numpy as np
import random
from scipy.stats import t as t_dist
import math

In [111]:
def fast_non_dominated_sort(pop_chrom, pop_cost):
    n_pop = len(pop_cost)
    # Rank of each candidate solution
    pop_rank = np.zeros(n_pop, dtype=int)
    # Set of solutions that solution p dominates
    S = [[] for _ in range(n_pop)]
    # Number of solutions that dominate solution p
    n = np.zeros(n_pop, dtype=int)
    # Pareto fronts
    F = [[]]
  
    for p_idx in range(n_pop):
        
        for q_idx in range(n_pop):
            if p_idx == q_idx:
                continue
         
            if dominates(pop_cost[p_idx], pop_cost[q_idx]):
           
                S[p_idx].append(q_idx)
            elif dominates(pop_cost[q_idx], pop_cost[p_idx]):
             
                n[p_idx] = n[p_idx] + 1
       
        if n[p_idx] == 0:
           
            pop_rank[p_idx] = 0
            F[0].append(p_idx)
  
    i = 0  # initialize the front counter
    while (F[i] != []):
      
        Q = []

        for p_idx in F[i]:
       
            for q_idx in S[p_idx]:
            
                n[q_idx] = n[q_idx] - 1
                
                if (n[q_idx] == 0):
      
                    pop_rank[q_idx] = i + 1
                    if q_idx not in Q:
                        Q.append(q_idx)
        i = i + 1
        F.append(Q)
    del F[len(F) - 1]

    return F, pop_rank

def dominates(x, y):
    if all(x <= y) and any(x < y):
        return 1
    else:
        return 0

In [112]:
def crowded_comparison_operator(p0_rank,p0_cdist, p1_rank, p1_cdist):
    if p0_rank < p1_rank:
        return 0
    if p1_rank < p0_rank:
        return 1
    if (p0_rank == p1_rank) and (p0_cdist > p1_cdist):
        return 0
    if (p0_rank == p1_rank) and (p1_cdist > p0_cdist):
        return 1
    return 0

In [113]:
def init_rv(M, N, lb, ub):
    pop_chrom = np.random.uniform(lb, ub, (M, N))
    pop_chrom[0] = lb
    pop_chrom[-1] = ub
    return pop_chrom

def zdt1(x):
    n_dim = len(x)
    f1 = x[0]  # objective 1
    g = 1 + 9 * np.sum(x[1:n_dim]) / (n_dim-1)
    h = 1 - np.sqrt(f1 / g)
    f2 = g * h  # objective 2
    return np.array([f1, f2])

In [114]:
def critical_values_t(N, gamma):
    # 1 - Number of degrees of freedom and probability p
    p       = (1 - gamma) / 2.0     # tail probability
    nDof    = N - 1                 # degrees of freedom

   
    pl = p
   
    cl = t_dist.ppf(pl, nDof)
    print('cl = {}'.format(cl))

    pu = 1 - p
    cu = t_dist.ppf(pu, nDof)
    print('cu = {}'.format(cu))

    return cl, cu

In [115]:

def sbx_crossover(p1,p2, pc,eta=2):
 
    n_dim = len(p1)
  
    if np.random.rand() < pc:
   
        c1 = np.zeros(n_dim)
        c2 = np.zeros(n_dim)

        for i in range(n_dim):
       
            u = random.random()
            if u <= 0.5:
                 beta = 2. * u
            else:
                 beta = 1. / (2. * (1. - u))
            beta = beta**(1. / (eta + 1.))
           
            c1[i] = 0.5 * (((1 + beta) * p1[i]) +
                            ((1 - beta) * p2[i]))
            c2[i] = 0.5 * (((1 - beta) * p1[i]) +
                            ((1 + beta) * p2[i]))
    else:
     
        c1 = p1
        c2 = p2
    return c1,c2

In [116]:
def mutation_polynomial_rv(x, pm, a, b, eta_m = 20):
  
    if pm == 0:
        return x
  
    r = np.random.uniform()
    if r < pm:
      
        n_var =len(x)
       
        mutated_x = np.copy(x)
     
        for k in range(n_var):
            u = np.random.uniform()
            if u <= 0.5:
                sig_L = (2 * u) ** (1 / (1+eta_m)) - 1
                mutated_x[k] = mutated_x[k] + \
                               sig_L*(mutated_x[k] - a[k])
            else:
                sig_R = 1 - (2 * (1 - u)) **  (1 / (1 + eta_m))
                mutated_x[k] = mutated_x[k] + \
                               sig_R * (b[k] - mutated_x[k])
    else:
      
        mutated_x = x
   
    return mutated_x

In [117]:


def binary_tournament(pop_chrom, pop_rank, pop_cdist):
    # number of chromosomes
    M = np.shape(pop_chrom)[0]
    # randomly select num_indvs individuals without replacement
    num_indvs = 3 
    inds = random.sample(range(1, M), num_indvs)
    # selected individuals
    selected_chroms = pop_chrom[inds]
    selected_ranks = pop_rank[inds]
    selected_cdists = pop_cdist[inds]

   
    nc = np.zeros(num_indvs, dtype=int)
    for i in range(num_indvs):
        p0_rank = selected_ranks[i]
        p0_cdist = selected_cdists[i]
        for j in range(i+1,num_indvs):
            p1_rank  = selected_ranks[j]
            p1_cdist =  selected_cdists[j]
        
            cco = crowded_comparison_operator(p0_rank, p0_cdist,
                                              p1_rank, p1_cdist)
         
            if cco == 0:
                nc[i] = nc[i] + 1
            else:
                nc[j] = nc[j] + 1

    # sort in descending order of victories
    inds_nc = np.argsort(-nc)
    # return the first two in number of victories
    ind_p1 = inds_nc[0]
    parent1 = selected_chroms[ind_p1]
    ind_p2 = inds_nc[1]
    parent2 = selected_chroms[ind_p2]

    return parent1, parent2

In [118]:
def crowding_distance(pop_cost, F, n_goals):
    n_fronts= len(F)
    
    pop_cdist= np.zeros(len(pop_cost))
    
    for idf in range(n_fronts):
        sol_indexes= F[idf]
        front_costs = pop_cost[sol_indexes]
        n_solutions =  len(sol_indexes)
        dist= np.zeros(n_solutions)
        
        for igd in range(n_goals):
            sorted_idx= np.argsort(front_costs[:,igd])
            cj= front_costs[sorted_idx,igd]
            dist[sorted_idx[0]]= math.inf
            dist[sorted_idx[-1]]= math.inf
            
            for ii in range(1, n_solutions -1):
                
                c_ini = cj[0]
                c_end= cj[-1]
                
                den = n_goals * np.abs(c_end - c_ini)
                num= np.abs(cj[ii+1]- cj[ii-1])
                if den == 0:
                    dist[sorted_idx[ii]] += num/den
                
        for idpop in range(n_solutions):
            pop_cdist[sol_indexes[idpop]]= dist[idpop]
            
    return pop_cdist
                
                
            
        

In [119]:
def nsga2(pop_size, max_iters,pc,pm,f_init,f_cost,f_cross,f_mut,lb,ub,n_dim,n_goals):
    if pop_size % 2 !=0 or pop_size<=2:
        raise ValueError("pop size must be even and more than 2")
    
    pop_chrom= f_init(pop_size, n_dim, lb, ub)
    
    pop_cost= np.zeros((pop_size, n_goals))
    for pop_index in range(pop_size):
        pop_cost[pop_index] = f_cost(pop_chrom[pop_index])
    
    F, pop_rank= fast_non_dominated_sort(pop_chrom,pop_cost)
    pop_cdist = crowding_distance(pop_cost, F, n_goals)
    
    for id_iter in range(max_iters):
        children_chrom = np.zeros((pop_size, n_dim))
        children_fit = np.zeros((pop_size, n_goals))
        
        for k in range(0, pop_size-1,2):
            [parent1_chrom,parent2_chrom]= binary_tournament(pop_chrom,pop_rank,pop_cdist)
            
            [child1_chrom,child2_chrom] = f_cross(parent1_chrom,parent2_chrom,pc)
            child1_chrom= np.clip(child1_chrom, lb,ub)
            child2_chrom= np.clip(child2_chrom, lb,ub)
            child1_chrom= f_mut(child1_chrom,pm, lb,ub)
            child2_chrom= f_mut(child2_chrom, pm, lb,ub)
            
            children_chrom[k]= child1_chrom
            children_chrom[k+1]= child2_chrom
            
            children_fit[k]= f_cost(children_chrom[k])
            children_fit[k+1]= f_cost(children_chrom[k+1])
            
        pop_comb_chrom = np.vstack((pop_chrom, children_chrom))
        pop_comb_fit = np.vstack((pop_cost, children_fit))
        
        [F,_]= fast_non_dominated_sort(pop_comb_chrom,pop_comb_fit)
        pop_comb_cdist = crowding_distance(pop_comb_fit, F, n_goals)
        
        inds_F= F[0]
        sorted_ids= np.argsort(-pop_comb_cdist[inds_F])
        sorted_ids= np.array(inds_F)[sorted_ids]
        sorted_front= pop_comb_chrom[sorted_ids]
        sorted_fit= pop_comb_fit[sorted_ids]
        new_pop_chrom=sorted_front
        new_pop_fit=sorted_fit
        n_fronts= len(F)
        for id_front in range(1, n_fronts):
            inds_F= F[id_front]
            if len(new_pop_chrom)<= pop_size:
                sorted_ids= np.argsort(-pop_comb_cdist[inds_F])
                sorted_ids= np.array(inds_F)[sorted_ids]
                sorted_front= pop_comb_chrom[sorted_ids]
                sorted_fit= pop_comb_fit[sorted_ids]
                new_pop_chrom=np.vstack((new_pop_chrom,sorted_front))
                new_pop_fit=np.vstack((new_pop_fit,sorted_fit))

                
        
        
        
        new_pop_chrom=new_pop_chrom[:pop_size]
        new_pop_fit=new_pop_fit[:pop_size]
        pop_chrom=np.copy(new_pop_chrom)  
        pop_cost=np.copy(new_pop_fit)  
        F, pop_rank= fast_non_dominated_sort(pop_chrom,pop_cost)
        pop_cdist = crowding_distance(pop_cost, F, n_goals)
        
        
    return pop_cost, pop_chrom    
        
        
        

In [ ]:
def calc_IDG_metric(true_pf, approx_PF):
    from scipy.spatial.distance import euclidean
    normZ = len(true_pf)
    normA = len(approx_PF)

    # IGD
    dist = np.zeros((normZ, normA))
    for idz in range(normZ):
        for ida in range(normA):
            dist[idz, ida] = euclidean(true_pf[idz], approx_PF[ida])
    idg = 0
    for idz in range(normZ):
        idg = idg + np.min(dist[idz])
    idg = (1 / normZ) * idg
    return idg

def generate_zdt1_pf():

    n_samples = 1000
   
    x1 = np.linspace(0, 1, n_samples)
    f1 = x1
   
    f2 = 1 - np.sqrt(f1)
    pareto_front = np.array([f1, f2]).T
    return pareto_front
# --------------------------------------------------------------
# Monte Carlo runs
# --------------------------------------------------------------
def perform_MC_runs(cost_f, true_pf, n_dim, n_goals, lb, ub, nMC):
 
    npop = 200            # Number of individuals in the population
    maxIterations = 500    # Maximum number of iterations

    #  Parameters for the algorithm
    pc          = 0.9        # Crossover probability
    pm          = 1/n_dim        # Mutation probability

    init_f      = init_rv                   # Creates the initial population
    crossover_f = sbx_crossover             # Implements crossover
    mutation_f  = mutation_polynomial_rv    # Implements mutation

  
    IGD         = np.zeros(nMC) # Store IGD values
    np.random.seed(123)                 # For repeatability
  
    for id_MC in range(nMC):   
        # Monte Carlo runs
     
        approx_PF, _  = nsga2(npop, maxIterations, pc, pm,
                      init_f,
                      cost_f,
                      crossover_f,
                      mutation_f,
                      lb, ub, n_dim, n_goals)

        IGD[id_MC] = calc_IDG_metric(true_pf, approx_PF)

    return IGD

sigma   = 0.95    # confidence level
N       = 40      # number of samples

#  Compute the critical values of the t distribution
[cl,cu] = critical_values_t(N,sigma)

#  Define the test problem and its optimal solution
f_name = 'ZDT1'
n_dim = 30
f_cost = zdt1
n_goals = 2
ubc = 1
lbc = 0
lb = lbc * np.ones(n_dim)  # Lower bound for the
# search space
ub = ubc * np.ones(n_dim)  # Upper bound for the
# search space
true_PF = generate_zdt1_pf()
#  Perform the Monte Carlo runs
X = perform_MC_runs(f_cost,
                       true_PF,
                       n_dim,
                       n_goals,
                       lb,
                       ub,
                       N)
print("hellocost")
#  Compute the sample mean and sample standard
#     deviation
Xbar = np.mean(X)   # sample mean
print('Xbar = {}'.format(Xbar))
S = np.std(X)       # sample sd
print('S = {}'.format(S))

#  Compute the confidence Interval
lb = Xbar - cu * (S / np.sqrt(N))  # lower bound of
                                   # the CI
ub = Xbar - cl * (S / np.sqrt(N))  # upper bound of
                                   # the CI
print('the {}% CI is [{},{}]'.format(sigma * 100, lb, ub))